In [ ]:
!pip install bing-image-downloader

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from bing_image_downloader import downloader as dw

In [ ]:
!mkdir train

In [ ]:
dw.download("Bacterial leaf Blight",limit=25,output_dir='/content/train',adult_filter_off=True,verbose=False)

[%] Downloading Images to /content/train/Bacterial leaf Blight


[!!]Indexing page: 1

[%] Indexed 25 Images on Page 1.


[%] Downloading Image #1 from http://1.bp.blogspot.com/-ua4TbkhHfp0/TdtB5eTVW8I/AAAAAAAAASQ/SPCICcUqESg/s1600/7falsesmut2.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://3.bp.blogspot.com/-oEMD9SvLd1I/TdtCa_zjxlI/AAAAAAAAASc/NkmQ2m-53nE/s1600/9BacterialLeafStreakRice.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://1.bp.blogspot.com/-p9anGASiNls/TdtCHU-CsDI/AAAAAAAAASU/P1j5jUYJoxY/s1600/8BacterialLeafBlightRice.jpg
[%] File Downloaded !

[%] Downloading Image #4 from http://juagrisciences.com/uploads/crop/1556364420bacterialleafblightblb.jpg
[%] File Downloaded !

[%] Downloading Image #5 from http://juagrisciences.com/uploads/crop/1556364567falsesmut.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https://journals.ashs.org/hortsci/view/journals/hortsci/53/6/full-838fig1.jpeg
[%] File Downloaded !

[%] Downloading Image #7 f

In [ ]:
dw.download("Alternaria Leaf Spot",limit=55,output_dir='/content/train',adult_filter_off=True,verbose=False)

[%] Downloading Images to /content/train/Alternaria Leaf Spot


[!!]Indexing page: 1

[%] Indexed 15 Images on Page 1.


[%] Downloading Image #1 from https://c8.alamy.com/comp/X62XXB/cercospora-leaf-spot-cercospora-beticola-infection-on-a-sugar-beet-leaf-X62XXB.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://journals.ashs.org/horttech/view/journals/horttech/26/4/full-542fig1.jpeg
[!] Issue getting: https://journals.ashs.org/horttech/view/journals/horttech/26/4/full-542fig1.jpeg
[!] Error:: HTTP Error 403: Forbidden
[%] Downloading Image #2 from https://bugwoodcloud.org/images/768x512/5372598.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://bugwoodcloud.org/images/768x512/5430722.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://2.bp.blogspot.com/-rFod5QKlST0/V-6BGdMY6LI/AAAAAAAABhc/YuQuFlotne4l1HPV6Oq7DjDN9gd7mjpUgCLcB/s1600/cercospora%2Bleaf%2Bspot%2Bin%2Bcastor.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://geneticlit

In [ ]:
dw.download("Gray Mildew",limit=55,output_dir='/content/train',adult_filter_off=True,verbose=True)

[%] Downloading Images to /content/train/Gray Mildew


[!!]Indexing page: 1

[%] Indexed 15 Images on Page 1.


[%] Downloading Image #1 from https://bugwoodcloud.org/images/1536x1024/2110097.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://bugwoodcloud.org/images/1536x1024/5528798.jpg
[%] File Downloaded !

[%] Downloading Image #3 from http://ucanr.edu/blogs/Alfalfa/blogfiles/34939_original.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://bugwoodcloud.org/images/1536x1024/5474075.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://www.gardentech.com/-/media/Images/GardenTech-NA/US/pest-id/plant-pest/Downy-mildew/Downy-mildew-onion.jpg?h=658&amp;la=en&amp;w=1000&amp;hash=459ADC4EF073B037F73C0618CC8CA33905C24575
[%] File Downloaded !

[%] Downloading Image #6 from https://bugwoodcloud.org/images/768x512/1495096.jpg
[%] File Downloaded !

[%] Downloading Image #7 from https://blogs.cornell.edu/ipmwpr/files/2014/07/032-2-22usd4y.jpg
[%] File D

In [ ]:
dw.download("Magnesium Deficiency",limit=55,output_dir='/content/train',adult_filter_off=True,verbose=True)

[%] Downloading Images to /content/train/Magnesium Deficiency


[!!]Indexing page: 1

[%] Indexed 15 Images on Page 1.


[%] Downloading Image #1 from https://rumorfix.com/wp-content/uploads/2020/10/magnesium-1024x1024.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://i.pinimg.com/474x/36/f7/c0/36f7c02d1dec4a234e03f2b3db5e4a86--magnesium-deficiency-symptoms-magnesium-foods.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://i.pinimg.com/originals/50/06/f1/5006f1e82a81e10105c462768ddae44d.png
[%] File Downloaded !

[%] Downloading Image #4 from https://i.pinimg.com/originals/e9/ad/cf/e9adcf9490e4bd1e1f591f3b0c20ec8f.png
[%] File Downloaded !

[%] Downloading Image #5 from https://www.glutenfreesociety.org/wp-content/uploads/2015/03/Magnesium-Deficiency-Symptoms.png
[%] File Downloaded !

[%] Downloading Image #6 from https://www.tandfonline.com/doi/cover-img/10.1080/ipgm20.v092.i05
[%] File Downloaded !

[%] Downloading Image #7 from https://i.pinimg.com/736x

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
        './train/',
        target_size=(150, 150))

Found 60 images belonging to 4 classes.


In [ ]:
image_batch, label_batch = next(iter(train_generator))

In [ ]:
bm=tf.keras.applications.vgg16.VGG16(input_shape=(150,150,3),include_top=False,  weights='imagenet')

In [ ]:
bm.trainable=False

In [ ]:
feature_batch = bm(image_batch)

In [ ]:
pl=tf.keras.layers.Dense(4)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
prediction_batch = pl(feature_batch_average)

In [ ]:
feature_batch_average.shape

TensorShape([60, 512])

In [ ]:
inp=tf.keras.Input(shape=(150,150,3))
x=bm(inp,training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
out = pl(x)
m=tf.keras.Model(inp,out)

In [ ]:
m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
label_batch = np.asarray(label_batch)
image_batch = np.asarray(image_batch)

In [ ]:
m.fit(image_batch,label_batch,epochs=10)

Epoch 1/10
2/2 [==============================] - 12s 5s/step - loss: 0.8780 - accuracy: 0.7500
Epoch 2/10
2/2 [==============================] - 11s 5s/step - loss: 0.9093 - accuracy: 0.7667
Epoch 3/10
2/2 [==============================] - 11s 5s/step - loss: 0.8424 - accuracy: 0.8000
Epoch 4/10
2/2 [==============================] - 11s 5s/step - loss: 0.8527 - accuracy: 0.8000
Epoch 5/10
2/2 [==============================] - 11s 5s/step - loss: 0.8820 - accuracy: 0.7333
Epoch 6/10
2/2 [==============================] - 11s 5s/step - loss: 0.8419 - accuracy: 0.8000
Epoch 7/10
2/2 [==============================] - 11s 5s/step - loss: 0.8111 - accuracy: 0.7833
Epoch 8/10
2/2 [==============================] - 11s 5s/step - loss: 0.8147 - accuracy: 0.7833
Epoch 9/10
2/2 [==============================] - 11s 5s/step - loss: 0.7957 - accuracy: 0.8000
Epoch 10/10
2/2 [==============================] - 11s 5s/step - loss: 0.7905 - accuracy: 0.7833


In [ ]:
m.save('model.h5')

In [ ]:
for i in dir(m):
  print(i)

_SCALAR_UPRANKING_ON
_TF_MODULE_IGNORED_PROPERTIES
__call__
__class__
__copy__
__deepcopy__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__getstate__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__setstate__
__sizeof__
__str__
__subclasshook__
__weakref__
_activity_regularizer
_add_trackable
_add_trackable_child
_add_variable_with_custom_getter
_assert_compile_was_called
_assert_weights_created
_auto_track_sub_layers
_autocast
_autographed_call
_base_model_initialized
_build_input_shape
_call_accepts_kwargs
_call_arg_was_passed
_call_fn_arg_defaults
_call_fn_arg_positions
_call_fn_args
_call_full_argspec
_callable_losses
_cast_single_input
_check_call_args
_checkpoint_dependencies
_clear_losses
_cluster_coordinator
_compile_was_called
_compiled_trainable_state
_compute_dtype
_compute_dtype_object
_compute_output_and_mask_jointly
_compute_tensor_usage_count
_configure_st